# LDA Topic Model of MALE Antivax+AntiAntivax Comments on Facebook

In [1]:
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import nltk

import os
from time import time
from collections import defaultdict

import numpy as np
import pandas as pd

import gensim
from gensim import corpora, utils
from gensim.corpora.dictionary import Dictionary
from gensim.models.wrappers.dtmmodel import DtmModel

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

import matplotlib
import matplotlib.pyplot as plt

from dateutil import parser
import datetime

import random

%matplotlib inline

In [2]:
import csv

#with open('REDDIT_DATASET_ONE_YEAR_COMMENTS.csv', 'r') as f:
#  reader = csv.reader(f)
#  your_list = list(reader)

# with open('REDDIT_DATASET_ONE_YEAR_COMMENTS_FEMALE_RELATED_COMMENTS.csv', 'r') as f:
#   reader = csv.reader(f)
#   your_list = list(reader)

with open('./data/combined_male_data.csv', 'r') as f:
   reader = csv.reader(f)
   your_list = list(reader)

# reader = pd.read_csv('./data/combined_male_data.csv', sep=',', encoding='latin-1')
# your_list = reader['x'].tolist()
    
your_list.pop(0) # remove the first element which is accidentally put there by R after exporting

['x']

In [3]:
print(your_list[:5]) # first 5 elements
# print(reader[:5]) # first 5 elements

[['Keep up the good work of sharing the awareness.'], ['Everybody shows some form of autistic trait!!!!!!'], ["Okay, while you are being condescending listen to this. The Autistic Spectrum is hugely broad! What I was trying to get across (and failed completely obviously) is that Autistic traits can be seen in all people yet it is the amount of traits that helps confirm a diagnosis. We are seeing a rise in the numbers of Autism diagnosis's due to more education and awareness of the disorder."], ["And I love the way you insult anyone who doesn't agree with you!"], ["You seem to have a very narrow view of what Autism is! Not all Autistic people are incapable of talking or using the toilet. I myself am a special education teacher who has case managed many autistic children and find them to be often rather more capable in many ways. We can't pigeon hole autism that way."]]


In [4]:
len(your_list)

307920

In [5]:
import itertools
merged_list = list(itertools.chain(*your_list))

In [6]:
# import random
# merged_list = merged_list[1:10000] # we create a small toy dataset to work with 
# merged_list = random.sample(merged_list, 100000)
# print(merged_list[:5]) # view first 5 elements

In [7]:
len(merged_list)

307920

In [8]:
comments_text = [comment for comment in merged_list]
comments_text_original = [comment for comment in merged_list]

In [9]:
# do word stemming?
from stemming.porter2 import stem

comments_text = [[stem(word) for word in sentence.split(" ")] for sentence in comments_text]
comments_text = [" ".join(sentence) for sentence in comments_text]

In [10]:
# print(comments_text[:5])

## Generate topic model

In this section we generate the topic model. There are some particular parameters that have an important role in shaping the outcome of the model. Changing these will affect the output in various ways. 

I have tested quite a few different combinations and the current output seems to give reasonable results (although we can certainly do better).

num_topics = 30 ... we specify that we want to generate 30 topics from our data
word stemming ... we use a word stemmer to reduce some words to their roots (e.g., walked and walking becomes "walk")
n-grams ... we specify that we want single terms (e.g. "walked"), and also find bi-grams or 2-grams (e.g., "she walked")
stop words ... we remove common English words that are not useful for analysis (e.g. "the", "and")
max_df ... we ignore terms that have a document frequency strictly higher than the given threshold
min_df ... ignore terms that have a document frequency strictly lower than the given threshold


In [11]:
# for good overview of these values see: http://stackoverflow.com/a/35615151/2589495
max_df = 0.05 # ignore terms that have a document frequency strictly higher than x% of docs (or occurs in > n docs if integer value)
min_df = 2 # ignore terms that have a document frequency strictly less than x% of documents (or occurs in < n documents if integer value)
# Some good values found so far...
# max_df = 0.05 (smaller in same order of magnititude is also good e.g. 0.03)
# min_df = 2
# k topics = 50
# passes = 3

# bi-grams:
# tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,ngram_range=(1,2),stop_words='english',analyzer='word')
# unigrams 
tf_vectorizer = CountVectorizer(max_df=max_df, min_df=min_df,stop_words='english')
tf = tf_vectorizer.fit_transform(comments_text)

In [12]:
# dictionary.filter_tokens(bad_ids=low_value_words)
# corpus = [dictionary.doc2bow(doc) for doc in texts]
corpus = gensim.matutils.Sparse2Corpus(tf.T)
n_terms, n_docs = corpus.sparse.shape
id2word = {i:word for i, word in enumerate(tf_vectorizer.get_feature_names())}
dictionary = Dictionary.from_corpus(corpus, id2word=id2word)

In [13]:
# generate LDA model
number_topics = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=number_topics, id2word = dictionary, passes=100,update_every=1, chunksize=10000)

In [14]:
# ldamodel.print_topics(-1)

## Visualisation of topics (by term/word)

This is an interactive visualisation of the topic model. The right-hand side shows which terms belong to each topic and how important they are for that topic (in order of importance). 

The left-hand Intertopic Distance Map gives an indication of how closely related the topics are (projected into a two-dimensional space). It may or may not be that helpful analytically, but is included anyway. 

In [15]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary,sort_topics=False)
pyLDAvis.save_html(vis, 'lda_FINAL_MODEL_FOR_PAPER.html')

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  topic_term_dists = topic_term_dists.ix[topic_order]


In [16]:
pyLDAvis.display(vis)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.632145. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [17]:
docTopicProbMat = ldamodel[corpus]

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.639667. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [18]:
docTopicProbMat_list = list(docTopicProbMat)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.640595. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [19]:
# print(docTopicProbMat_list[7]) # get the topic probabilities for a document
# print(comments_text[7]) # and the document text

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.641599. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [20]:
comment_probabilities = []
for doc in docTopicProbMat_list:
    docTopic = 999 # reset the topic
    bestMatch = 0 # reset the best match
    possibleMatch = 0 # reset the possible match
    for topic in doc:
        possibleMatch = topic[1]
        if possibleMatch > bestMatch:
                bestMatch = topic[1]
                if bestMatch > 0.6: # minimum threshold for comment topic relatedness
                    docTopic = topic[0]
                # print("bestMatch is now: ",bestMatch)
                # print("docTopic is now: ",docTopic)
    # print("best topic was: ",docTopic)
    comment_probabilities.append(docTopic)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.642651. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [21]:
# comment_probabilities[0:10]

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.643578. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [22]:
comment_probabilities = [probs + 1 for probs in comment_probabilities] # now when we ask for topic n we actually get topic n (not topic n-1)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.644754. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [23]:
# print(comments_text[8])
# comment_probabilities[8]

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.645870. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [24]:
comments_and_topics = pd.DataFrame(
    {'original_comment_text': comments_text_original,
     'comment_text': comments_text,
     'topic': comment_probabilities
    })

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.646859. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [25]:
# comments_and_topics

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.647921. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


In [26]:
# a function to get n random example comments assigned to a given topic k
def getTopicExamples(topic_id,num_examples):
    topic_comments = list(comments_and_topics['original_comment_text'][comments_and_topics.topic == topic_id])
    topic_comments = random.sample(topic_comments, num_examples)
    return(topic_comments)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.648961. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


## Extract 10 comment examples for each topic

In this section we extract 50 (randomly selected) text comments that belong to each topic. 

This provides some examples of comments that belong to each topic. The topics are ordered from 1 to k (where k is the number of topics we specified for the model). 

You can use these comment examples to help make sense of the topic terms in the above visualisation, in other words "what is going on" for each topic.

In [27]:
for k in range(1,number_topics+1):
    topic_example_comments = getTopicExamples(k,50)
    print("\n____________________ TOPIC",k,"______________________________\n")
    print(*topic_example_comments, sep='\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - \n')

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.651565. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)



____________________ TOPIC 1 ______________________________

Jen Wennerberg
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Probably chemtrails.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
And chemtrails, don't forget chemtrails.  ;)
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
She was an idiot to be an anti-vaxer in the first place.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Norman Borlaug.
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Penn and Teller?  The magicians?
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Shaylah Maree
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
They are pro death
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
There are just countless examples
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Shaylah Maree
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
Scare tactics
- - - - - - - - - - - - - - - - - - - - - - - - - - - - 
What are you on about? Vaccin

In [29]:
f1=open('./example_comments_numComments50_numTopics30_LDA_MODEL_A.txt', 'w+')
for k in range(1,number_topics+1):
    topic_example_comments = getTopicExamples(k,50)
    f1.write("\n____________________ TOPIC",k,"______________________________\n")
    f1.write(*topic_example_comments, sep='\n- - - - - - - - - - - - - - - - - - - - - - - - - - - - \n')

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.655601. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)


TypeError: write() takes exactly one argument (3 given)

/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.656735. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.657767. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.658983. Please add timezone info to timestamps.
  new_obj[k] = extract_dates(v)
/Users/Tim/anaconda/envs/reddit_data_topic_model/lib/python3.6/site-packages/jupyter_client/jsonutil.py:67: DeprecationWarning: Interpreting naive datetime as local 2017-07-13 22:34:14.660349. Please add t

In [ ]:
# theta, _ = ldamodel.inference(corpus) # we can extract the thetas directly from the corpus

In [ ]:
# theta[8] # columns are indexed from 0, so topic n is topic n+1 (e.g. topic22 is topic23)

## Word2vec analysis of text

In this section we do some exploratory text analysis using the novel word2vec algorithm.

This is very experimental... just some computational doodling.

In [ ]:
sentences = [comment for comment in merged_list]
sentences = [[stem(word) for word in sentence.split(" ")] for sentence in sentences]

In [ ]:
# import nltk
# nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
filtered_words = [word for word in sentences if word not in stopwords.words('english')]

In [ ]:
# print(filtered_words[2]) # sanity check the data first

In [ ]:
# convert to lowercase
filtered_lower_words = []
for doc in filtered_words:
    filtered_lower_words.append([word.lower() for word in doc])

In [ ]:
# remove punctuation
import string
filtered_lower_words_nopunct = []
for doc in filtered_lower_words:
    filtered_lower_words_nopunct.append([''.join(c for c in s if c not in string.punctuation) for s in doc])

In [ ]:
# remove empty words due to previous steps 
filtered_lower_words_nopunct_nospaces = []
for doc in filtered_lower_words_nopunct:
    filtered_lower_words_nopunct_nospaces.append([s for s in doc if s])

In [ ]:
# print(filtered_lower_words_nopunct_nospaces[2]) # sanity check the data quickly

In [ ]:
from gensim.models import Word2Vec
min_count = 50
size = 120
window = 7
workers = 6
 
model = Word2Vec(filtered_lower_words_nopunct_nospaces, min_count=min_count, size=size, window=window)


Let's try the classic word2vec example:

King - man + woman = ?

In [ ]:
model.wv.most_similar(positive=['woman', 'king'], negative=['man'])

Using this kind of analysis we can ask other questions more specific to the paper:
    
Gamer - man + woman = ?

(What is 'gamer' without taking into account 'men', but taking into account 'women'?)

In [ ]:
model.wv.most_similar(positive=['women', 'gamer'], negative=['men'])

And we can also do the converse for men:

Gamer - woman + man = ?

(What is 'gamer' without taking into account women, but taking into account men?)

In [ ]:
model.wv.most_similar(positive=['men', 'gamer'], negative=['women'])

In [ ]:
model.most_similar("angry")

In [ ]:
model.most_similar("game")

In [ ]:
model.most_similar(["sjw"])

We can also see which word within a set of words doesn't belong:

In [ ]:
model.wv.doesnt_match("ps4 xbox pc chess".split())


Can the model differentiate female vs male characters within a game (Overwatch)?

In [ ]:
model.wv.doesnt_match("tracer genji widowmaker".split())


In [ ]:
model.wv.doesnt_match("chick hot cosplay guy".split())